In [2]:
import pandas as pd
from pathlib import Path
import calendar as cd
import re
from control import kp_deci, kp_in, busca1

In [4]:
colunas = ['tempo',0, 3, 6, 9, 12, 15, 18, 21]

kp_arqui = [arq for arq in Path('dados/kp').glob('*.txt') if 'limpo' not in str(arq)]

for caminho in kp_arqui:
    ano = int(''.join(busca1.findall(str(caminho))))
    df = pd.read_fwf(caminho, skiprows=0)
    mes_dia = [cd.monthrange(year=ano, month=i)[1] for i in range(1, 13)]
    q_dias_m = sum(mes_dia)
    #anos = [2005, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
    try:
        kp = df['Kp[8]           Sum'].str.split('', n=0, expand=True)
    except:
        kp = df['Kp[8]           Sum ap[8]'].str.split('', n=0, expand=True)

    df365 = []
    df365 = pd.DataFrame().T
    df365['tempo'] = pd.date_range(f'1/1/{ano}', periods=q_dias_m, freq='D')
    for num, simb, hora in zip(range(1, 16, 2), range(2, 17, 2), range(0, 22, 3)):
        df365[f'{hora}'] = kp[num]+kp[simb]
        

        
    df = pd.melt(df365, id_vars='tempo', var_name='hora', value_name='kp', ignore_index = True).sort_values('tempo')
    df.reset_index(inplace=True)
    df.drop(['tempo', 'index'], axis=1, inplace=True)
    df['hora'] = df.hora.astype(int)


    start1 = 0
    start2 = 8
    fim = len(df)
    while start2<=fim:
        df[start1:start2] = df[start1:start2].sort_values('hora', ascending=True)
        start1 = start2
        start2+=8
    df.set_index(pd.date_range(start=f'{ano}/1/1', periods=8*q_dias_m, freq='3h'), inplace=True)
    #df.drop('hora', axis=1, inplace=True)
    df['Kp_o'] = df['kp']
    for ki, kd in zip(kp_in, kp_deci):
        df['kp'].replace(ki, kd, inplace=True)
    df.kp.astype(float)
    df.to_string(f'dados/kp/kp{ano}_limpo.txt', index=True)

    